# Trabajo Práctico 1 Parte 2

## Python

In [ ]:
import random
import pprint
import threading

TAM = 5

a = []
b = []
cs = []  # secuencial
cc = []  # concurrente

def suma_filas(num_fila: int):
    """
    :param num_fila: entre 0 y 4 para este caso
    :return: None
    """
    for i in range(TAM):
        cc[num_fila][i] = a[num_fila][i] + b[num_fila][i]


# carga de valores en matrices
for i in range(TAM):
    a.append([random.randint(-32, 32) for _ in range(TAM)])
    b.append([random.randint(-32, 32) for _ in range(TAM)])
    cc.append([0 for _ in range(TAM)])  # La creo con valores para tener la estructura y poder hacer cc[x][y]

# algoritmo secuencial
for i in range(TAM):
    fila_nueva = []
    for j in range(TAM):
        fila_nueva.append(a[i][j] + b[i][j])
    cs.append(fila_nueva)

# algoritmo concurrente


hilos = [threading.Thread(target=suma_filas, args=(i,), name=f"suma_fila {i}") for i in range(TAM)]

for i in hilos:
    i.start()
    i.join()

# Python tiene algo llamado GIL (global interpreter lock), que significa solo un hilo puede estar haciendo operaciones.
# El beneficio de performance de tener threads en python solo se ve cuando está esperando por IO o por una conexion,
# para estos casos donde es solo hacer operaciones no hay diferencia


print("Matriz A:")
pprint.pp(a, )
print("\nMatriz B:")
pprint.pp(b)
print("\nMatriz CS (Secuencial):")
pprint.pp(cs)
print("\nMatriz CC (Concurrente):")
pprint.pp(cc)
print(f"\nLas matrices CC y CS son iguales? {cc == cs}")

Matriz A:
[[22, 22, -22, -18, 1],
 [-26, -30, -3, 27, -24],
 [0, -8, 14, 11, 18],
 [-23, -6, 20, 29, 12],
 [3, 20, -17, -20, -5]]

Matriz B:
[[-27, 10, 15, 29, 15],
 [21, 6, -13, 2, 9],
 [20, -27, -3, -27, 14],
 [28, 1, -18, 20, -18],
 [27, -20, 9, -12, 3]]

Matriz CS (Secuencial):
[[-5, 32, -7, 11, 16],
 [-5, -24, -16, 29, -15],
 [20, -35, 11, -16, 32],
 [5, -5, 2, 49, -6],
 [30, 0, -8, -32, -2]]

Matriz CC (Concurrente):
[[-5, 32, -7, 11, 16],
 [-5, -24, -16, 29, -15],
 [20, -35, 11, -16, 32],
 [5, -5, 2, 49, -6],
 [30, 0, -8, -32, -2]]

Las matrices CC y CS son iguales? True


## Java:

In [ ]:
%%writefile Process.java
import java.util.Random;

public class MatricesAleatorias
{
    private static int rows = 5;
    private static int columns = 5;
    private static int max = 32;
    private static int min = -32;

    public static void main(String[] args) throws InterruptedException
    {

        int[][] matrixA = generateRadomMatrix(rows, columns, min, max);
        int[][] matrixB = generateRadomMatrix(rows, columns, min, max);
        int[][] matrixC = addArraysSecuencial(matrixA,matrixB);
        int[][] matrixD = addArraysWithThreads(matrixA,matrixB);

        System.out.println("matrix A:");
        printMatrix(matrixA);

        System.out.println("\nmatrix B:");
        printMatrix(matrixB);

        boolean matrixAreEquals = arraysAreEquals(matrixC,matrixD);
        if(matrixAreEquals)
        {
            System.out.println("\nmatrix CC:");
            printMatrix(matrixD);
        }
        else
        {
            System.out.println("\nLas matrices no son iguales.");
        }

    }


    public static int[][] generateRadomMatrix(int rows, int columns, int min, int max)
    {
        int[][] matrix = new int[rows][columns];
        Random rand = new Random();

        for (int i = 0; i < rows; i++)
        {
            for (int j = 0; j < columns; j++)
            {
                matrix[i][j] = rand.nextInt((max - min + 1)) + min;
            }
        }

        return matrix;
    }

    public static void printMatrix(int[][] matrix)
    {
        int padding = 5;
        for (int i = 0; i < matrix.length; i++)
        {
            for (int j = 0; j < matrix[i].length; j++)
            {
                String number = String.format("%" + padding + "d", matrix[i][j]);
                System.out.print(number + " ");
            }
            System.out.println();
        }
    }

    public static int[][] addArraysSecuencial(int[][] matrixA, int[][] matrixB)
    {
        int rows = matrixA.length;
        int columns = matrixA[0].length;
        int[][] result = new int[rows][columns];

        for (int i = 0; i < rows; i++)
        {
            for (int j = 0; j < columns; j++)
            {
                result[i][j] = matrixA[i][j] + matrixB[i][j];
            }
        }

        return result;
    }

    public static int[][] addArraysWithThreads (int[][]matrixA, int[][] matrixB) throws InterruptedException
    {
        int[][] arrayAdd = new int[rows][columns];

        Thread[] threads = new Thread[rows];

        for (int i = 0; i < rows; i++)
        {
            final int row = i;
            threads[i] = new Thread(() ->
            {
                for (int j = 0; j < columns; j++)
                {
                    arrayAdd[row][j] = matrixA[row][j] + matrixB[row][j];
                }
            }
            );
            threads[i].start();
        }

        for (Thread thread : threads)
        {
            try
            {
                thread.join();
            }
            catch (InterruptedException e)
            {
                e.printStackTrace();
            }
        }
        return arrayAdd;
    }

     public static boolean arraysAreEquals(int[][] matrixA, int[][] matrixB)
     {
        if (matrixA.length != matrixB.length || matrixA[0].length != matrixB[0].length)
        {
            return false;
        }
        for (int i = 0; i < matrixA.length; i++)
        {
            for (int j = 0; j < matrixA[0].length; j++)
            {
                if (matrixA[i][j] != matrixB[i][j])
                {
                    return false;
                }
            }
        }
        return true;
    }
}

Writing Process.java


In [ ]:
!java Process.java

matrix A:
  -20     7   -23    32   -11 
  -22   -16    -3   -20   -12 
   22     9   -12    -5   -24 
  -25   -11     6    -5   -28 
   16     2   -18   -26    -3 

matrix B:
    3   -32   -23     9     6 
   16   -29     6   -18    20 
   11    -3    -4    22    -4 
    0     0     2    23    19 
  -13   -18    -2   -14    18 

matrix CC:
  -17   -25   -46    41    -5 
   -6   -45     3   -38     8 
   33     6   -16    17   -28 
  -25   -11     8    18    -9 
    3   -16   -20   -40    15 


## C++

In [ ]:
%%writefile matrix.cpp
#include <thread>
#include <iostream>
#include <cstdlib>
#include <ctime>

#define MAX 32
#define MIN -32
#define ROWS 5
#define COLUMNS 5
#define THREADS 5

int GenerateRandomNumber(int min, int max);
int** GenerateMatrix(int rows, int columns);
void PrintMatrix(int **matrix, int rows, int columns, std::string name);
void FillMatrix(int** matrix, int rows, int columns);
void SumSecuencial(int** A, int** B, int** result, int rows, int columns);
void SumConcurrent(int** A, int** B, int** result, int rows, int columns, int threads);
void CompareMatrix(int **A, int **b, int rows, int columns);
void SumRow(int* A, int* B, int* result, int columns);
void FreeMatrix(int **matrix, int rows);

int main(int argc, char *argv[])
{
    srand(time(NULL));

    int **A = GenerateMatrix(ROWS, COLUMNS);
    int **B = GenerateMatrix(ROWS, COLUMNS);
    int **CS = GenerateMatrix(ROWS, COLUMNS);
    int **CC = GenerateMatrix(ROWS, COLUMNS);

    FillMatrix(A, ROWS, COLUMNS);
    FillMatrix(B, ROWS, COLUMNS);
    SumSecuencial(A, B, CS, ROWS, COLUMNS);
    SumConcurrent(A, B, CC, ROWS, COLUMNS, THREADS);

    PrintMatrix(A, ROWS, COLUMNS, "A");
    PrintMatrix(B, ROWS, COLUMNS, "B");
    PrintMatrix(CS, ROWS, COLUMNS, "CS");
    PrintMatrix(CC, ROWS, COLUMNS, "CC");

    CompareMatrix(CC, CS, ROWS, COLUMNS);

    FreeMatrix(A, ROWS);
    FreeMatrix(B, ROWS);
    FreeMatrix(CC, ROWS);
    FreeMatrix(CS, ROWS);
}


int** GenerateMatrix(int rows, int columns)
{
   int **result = (int **) calloc(rows, sizeof (int *) * rows);
   for(int i = 0; i < rows; i++)
   {
     result[i] = (int *) calloc(columns, sizeof (int) * columns);
   }
   return result;
}


void FillMatrix(int** matrix, int rows, int columns)
{
  for(int i = 0; i < rows; i++)
     for(int j= 0; j < columns; j++)
       matrix[i][j] = GenerateRandomNumber(MIN, MAX);
}


int GenerateRandomNumber(int min, int max)
{
   int mod = (((max) + 1)-(min));
   int r = rand() % mod + min;
   return r;
}


void PrintMatrix(int **matrix, int rows, int columns, std::string name)
{
     std::cout<<"\n\nMatrix "<<name<<":\n\n"<<std::endl;
     for(int i = 0; i < rows; i++)
     {
       for(int j= 0; j < columns; j++)
         printf(" %d ", matrix[i][j]);
       printf("\n");
     }
}


void SumSecuencial(int** A, int** B, int** result, int rows, int columns)
{
  for(int i = 0; i < rows; i++)
    for(int j = 0; j < columns; j++)
      result[i][j] = A[i][j] + B[i][j];
}


void SumConcurrent(int** A, int** B, int** result, int rows, int columns, int threads)
{
  std::thread v_th[threads];
  for(int i = 0; i < threads; i++)
  {
    v_th[i] = std::thread(SumRow, A[i], B[i], result[i], columns);
  }
  for(int i = 0; i < threads; i++)
  {
    v_th[i].join();
  }
}


void SumRow(int* A, int* B, int* result, int columns)
{
  for(int i = 0; i < columns; i++)
    result[i] = A[i] + B[i];
}


void FreeMatrix(int **matrix, int rows)
{
  for(int i = 0; i < rows; i++)
  {
    free(matrix[i]);
  }
  free(matrix);
}


void CompareMatrix(int **A, int **B, int rows, int columns)
{
  int iguales = 1;
  for(int i = 0; i < rows; i++)
    for(int j = 0; j < columns; j++)
      if(A[i][j] != B[i][j])
      {
         iguales = 0;
         i = rows;
         j = columns;
      }
  if(iguales)
  {
    printf("\n\nLas matrices CC y CS son IGUALES");
  }
  else
  {
    printf("\n\nLas matrices CC y CS son DISTINTAS");
  }
}

Writing matrix.cpp


In [ ]:
!g++ -o matrix.bin matrix.cpp

In [ ]:
!./matrix.bin



Matrix A:


 -24  -18  -22  11  14 
 12  -29  10  29  21 
 1  -16  -14  13  -19 
 5  22  32  18  -6 
 -1  17  -5  5  16 


Matrix B:


 -31  16  -13  9  -2 
 13  19  15  25  -2 
 -4  4  3  -27  3 
 -8  8  21  10  -12 
 -29  -16  -21  -30  -31 


Matrix CS:


 -55  -2  -35  20  12 
 25  -10  25  54  19 
 -3  -12  -11  -14  -16 
 -3  30  53  28  -18 
 -30  1  -26  -25  -15 


Matrix CC:


 -55  -2  -35  20  12 
 25  -10  25  54  19 
 -3  -12  -11  -14  -16 
 -3  30  53  28  -18 
 -30  1  -26  -25  -15 


Las matrices CC y CS son IGUALES

## Go

### Instalación de Go

In [ ]:
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!mkdir /usr/local/share/jupyter/kernels/gophernotes
!cp ~/go/src/github.com/gopherdata/gophernotes/kernel/* \
       /usr/local/share/jupyter/kernels/gophernotes

Repository: 'deb https://ppa.launchpadcontent.net/longsleep/golang-backports/ubuntu/ jammy main'
Description:
Golang 1.8, 1.9, 1.10, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.20 and 1.21 PPA for Ubuntu
More info: https://launchpad.net/~longsleep/+archive/ubuntu/golang-backports
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/longsleep-ubuntu-golang-backports-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/longsleep-ubuntu-golang-backports-jammy.list
Adding key to /etc/apt/trusted.gpg.d/longsleep-ubuntu-golang-backports.gpg with fingerprint 52B59B1571A79DBC054901C0F6BC817356A3D45E
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRel

In [ ]:
!%cd /content/go
!go mod init go

/bin/bash: line 1: fg: no job control
go: creating new go.mod: module go
panic: go: internal error: missing go root module

goroutine 1 [running]:
cmd/go/internal/modload.mustHaveGoRoot(...)
	cmd/go/internal/modload/buildlist.go:104
cmd/go/internal/modload.newRequirements(0x60?, {0xc00006c2c0?, 0x9abc60?, 0xc0000101e0?}, 0xc0001257e8?)
	cmd/go/internal/modload/buildlist.go:118 +0x6ab
cmd/go/internal/modload.updatePrunedRoots({0xb854b0, 0xf21f60}, 0x81e140?, 0xc0000780f0, {0x0, 0x0, 0xc000012034?}, {0x0, 0x0, 0x0}, ...)
	cmd/go/internal/modload/buildlist.go:1223 +0xc59
cmd/go/internal/modload.updateRoots({0xb854b0?, 0xf21f60?}, 0xb7d862?, 0x6?, {0x0?, 0x1?, 0xc000012a40?}, {0x0, 0x0, 0x0}, ...)
	cmd/go/internal/modload/buildlist.go:783 +0x9d
cmd/go/internal/modload.CreateModFile({0xb854b0, 0xf21f60}, {0x7ffdf04e1d5d, 0x2})
	cmd/go/internal/modload/init.go:1041 +0x8ef
cmd/go/internal/modcmd.runInit({0xb854b0?, 0xf21f60?}, 0xc00002e648?, {0xc0000220b0?, 0x9b49a0?, 0xa83b99?})
	cmd/go/inte

### Código

In [ ]:
%%writefile matrix.go
package main

import (
	"fmt"
	"math/rand"
	"sync"
)

const SIZE int = 5

var a [SIZE][SIZE]int
var b [SIZE][SIZE]int
var sc [SIZE][SIZE]int
var ss [SIZE][SIZE]int

func main() {
	for row := 0; row < SIZE; row++ {
		for column := 0; column < SIZE; column++ {
			a[row][column] = rand.Intn(64) - 32
			b[row][column] = rand.Intn(64) - 32
		}
	}
	printMatrix(a, "A")
	printMatrix(b, "B")

	concurrentSum()
	sequentialSum()

	printMatrix(sc, "SC")
	printMatrix(ss, "SS")
}

func sequentialSum() {
	for row := 0; row < SIZE; row++ {
		for column := 0; column < SIZE; column++ {
			ss[row][column] = a[row][column] + b[row][column]
		}
	}
}

func concurrentSum() {
	var wg sync.WaitGroup
	for row := 0; row < SIZE; row++ {
		wg.Add(1)
		go rowSum(row, &wg)
	}
	wg.Wait()

}

func rowSum(row int, wg *sync.WaitGroup) {
	defer wg.Done()
	for column := 0; column < SIZE; column++ {
		sc[row][column] = a[row][column] + b[row][column]
	}
}

func printMatrix(matrix [SIZE][SIZE]int, name string) {
	fmt.Printf("Matrix %s:\n", name)
	for row := 0; row < SIZE; row++ {
		for column := 0; column < SIZE; column++ {
			fmt.Printf("%3d\t", matrix[row][column])
		}
		fmt.Println("")
	}
	fmt.Println("")
}




Overwriting matrix.go


### Ejecución

In [ ]:
!go run matrix.go

Matrix A:
  9	 -2	 -4	 -7	 -2	
  4	 24	-29	 18	-19	
-30	-14	-30	  8	 -6	
 30	-15	-32	 12	 14	
 14	  2	 22	 23	-23	

Matrix B:
 10	 12	 -5	 19	-23	
 22	 22	-26	 11	 -6	
 27	  8	 27	 -9	  2	
-23	  9	-13	 -8	 17	
  1	-29	-30	 30	 14	

Matrix SC:
 19	 10	 -9	 12	-25	
 26	 46	-55	 29	-25	
 -3	 -6	 -3	 -1	 -4	
  7	 -6	-45	  4	 31	
 15	-27	 -8	 53	 -9	

Matrix SS:
 19	 10	 -9	 12	-25	
 26	 46	-55	 29	-25	
 -3	 -6	 -3	 -1	 -4	
  7	 -6	-45	  4	 31	
 15	-27	 -8	 53	 -9	

